In [1]:
import torch

In [2]:

torch.manual_seed(3)
batch_size = 10000
x = torch.rand(batch_size, 8, 1)
w_base = torch.rand(1, 8, 16)



In [2]:

def calc_chosen(x, w):

    inner = torch.min(x, w)
    y = torch.max(inner, keepdim=True, dim=-2)[0]
    chosen = (y == inner)
    return y, chosen


y_base, chosen_base = calc_chosen(x, w_base)

NameError: name 'x' is not defined

In [3]:
def update_rel(x, t, chosen=None, dim=0):

    negatives = torch.relu(x - t)
    positives = torch.min(x, t)
    if chosen is not None:
        positives = chosen * positives

    return (
        positives.sum(dim=dim, keepdim=True) / 
        (positives.sum(dim=dim, keepdim=True) + negatives.sum(dim=dim, keepdim=True))
    )


In [2]:

# 

chosen_rel = None
for i in range(5):
    w_rel = update_rel(x, y_base, chosen_rel)
    y_rel, chosen_rel = calc_chosen(x, w_rel)

    true_count = (
        (chosen_rel == True) & (chosen_base == True)
    ).float()
    fp_count = (
        (chosen_rel == False) & (chosen_base == True)
    ).float()
    print(
        'Rate: ',
        true_count.sum() /
        (fp_count + true_count).sum()
    )
    print(
        'Loss: ', (y_rel - y_base).abs().mean()
    )
# for the first round i 

# increases to roughly 80% accuracy on the second iteration
# indicates it should help to do an extra iteration

# on the first round do not use chosen

# inner_rel = torch.min(x, w_rel)
# y_rel = inner_rel.max(dim=-2, keepdim=True)
# chosen = (y_rel == inner_rel)

# how many of the indices match

NameError: name 'update_rel' is not defined

Looks like it is worth it to use rel..

1. Don't know how much it will help in a realistic case but it looks quite promising here
2. It's possible the straight through estimator is also doing as good as it can for multiple layers and that I need to add noise
3. When there is not a clear optimum. Maybe it does not matter too much
4. The loss decreases drastically on the second iteration. Regular gradient descent might get me the rest of the way
5. Not sure how things will be affected by using both x_rel and w_rel. But doing just one seems to help dramatically


For now... Implement a simpler version of the "second order version"


a. Use rel loss 2
b. gradually anneal it
c. get rid of it

d. How to deal with the regular intersection / Union?
 a. Use STE (?)
 b. Do not use it 
 c. Use the "soft version"

In [2]:
from rel import (
    train, MinMax, MinMaxTPLearner, 
    MinMaxLoss, MinMaxLearner, TruncatedGaussianNoise, 
    MaxMinLearner, MaxMin, RandNoise, SmoothSTEMaxMin, SmoothSTEMinMax
)

import torch.nn as nn
import torch.utils.data
import numpy as np
import zenkai
from itertools import chain

torch.manual_seed(3)
batch_size = 10000
x = torch.rand(batch_size, 32)

x_weight = 1.0
w_weight = 0.01
n_epochs = 200

neuron_base = MaxMin(
    32, 64
)
neuron_base2 = MinMax(
    64, 64
)

# neuron1 = SmoothSTEMinMax(
#     32, 64, a=10, adjust=False
# )
# neuron2 = SmoothSTEMinMax(
#     64, 64, a=10, adjust=False
# )
# neuron1 = MinMaxTPLearner(
#     32, 64, reduction='sum', rel_reduction='mean',
#     x_weight=0.01, w_weight=0.01 #, a=5
# )
# neuron2 = MinMaxTPLearner(
#     64, 64, reduction='sum', rel_reduction='mean', 
#     x_weight=0.01, w_weight=0.01 #, a=5
# )

neuron1 = MaxMinLearner(
    32, 64, reduction='sum', rel_reduction='mean',
    x_weight=0.01, w_weight=0.01
)
neuron2 = MinMaxLearner(
    64, 64, reduction='sum', rel_reduction='mean', 
    x_weight=0.01, w_weight=0.01
)

# neuron1 = SmoothSTEMaxMin(
#     32, 64, adjust=False, a=4
# )
# neuron2 = SmoothSTEMinMax(
#     64, 64, adjust=False, a=4
# )

noiser = nn.Dropout(0.2)
with torch.no_grad():
    t = neuron_base2(neuron_base(x))

dataset = torch.utils.data.TensorDataset(x, t)
optim = torch.optim.Adam(chain(neuron1.parameters(), neuron2.parameters()), lr=1e-3)

j = 0
for i in range(n_epochs):
    results = []
    for x_i, t_i in torch.utils.data.DataLoader(
        dataset, 128, True
    ):
        y_i = neuron1(x_i)
        # y_i = torch.clamp(
        #     y_i + torch.randn_like(y_i) * 0.01, 0, 1
        # )
        y_i = neuron2(y_i)
        # loss = min_max_loss(x_i, y_i, t_i)
        loss = (y_i - t_i).pow(2).mean()
        # print(
        #     loss.item(),
        #     (y_i - t_i).pow(2).sum() / y_i.numel()
        # )
        optim.zero_grad()
        # reg2 = 1e-6 * (1 - neuron2.weight).abs().mean()
        # reg1 = 1e-6 * (neuron1.weight).abs().mean()
        
        reg2 = 1e-6 * (1 - neuron2.min_max.weight).abs().mean()
        reg1 = 1e-6 * (neuron1.max_min.weight).abs().mean()
        # (loss + reg2 + reg2).backward()
        loss.backward()
        optim.step()
        zenkai.params.apply_p(
            neuron1, lambda p: torch.clamp(p, 0.0, 1.0) 
        )
        zenkai.params.apply_p(
            neuron2, lambda p: torch.clamp(p, 0.0, 1.0) 
        )
        results.append(loss.item())
        # results.append(loss.item() / y_i.numel())
    
    # if i % 2 == 0:
    j += 1
    if (j >= 150):
        neuron1.a = None
        neuron2.a = None
    elif (j % 10) == 0:
        # neuron1.w_weight = 0.0 # neuron1.w_weight * 0.5
        # neuron2.w_weight = 0.0 # neuron2.w_weight * 0.5
        neuron1.a = neuron1.a + 2
        neuron2.a = neuron2.a + 2
    
    # else:
    #     neuron1.w_weight = 0.0 # neuron1.w_weight * 0.5
    #     neuron2.w_weight = 0.01 # neuron2.w_weight * 0.5

        # j = 0
    
    # neuron1.w_weight = neuron1.w_weight * 0.95
    # neuron2.w_weight = neuron2.w_weight * 0.95
    # neuron2.x_weight = neuron2.x_weight * 0.95
    # neuron1.a = neuron1.a + 1
    #neuron2.a = neuron2.a + 1
    # else:
    #     neuron1.w_weight = neuron1.w_weight * 0.95
    #     neuron2.w_weight = 0.0
    #     neuron2.x_weight = 0.1

    print(f'{i}: {np.mean(results)}')



0: 0.003598089571570671
1: 0.0019794574222539232
2: 0.0017511042106094993
3: 0.0016865623846673702
4: 0.0016601198526759503
5: 0.001626538789147346
6: 0.0015927994214116207
7: 0.001574100762947545
8: 0.0015559627665678345
9: 0.0015457969105837843
10: 0.001954009567823591
11: 0.0015383411576099034
12: 0.0015190490285712707
13: 0.0014959410557971347
14: 0.001476551691966155
15: 0.001453648162180487
16: 0.001443045698636789
17: 0.001414831922259889
18: 0.00140547555358491
19: 0.001388792095918067
20: 0.001439365626112381
21: 0.0012935908862000591
22: 0.0012628046657206327
23: 0.0012440436616940778
24: 0.001227161038778958
25: 0.0012112650769774483
26: 0.001187106413189086
27: 0.0011762844153998208
28: 0.0011760871341640624
29: 0.0011537875692784504
30: 0.0011445264143374147
31: 0.0010901195987398865
32: 0.0010834471352068307
33: 0.0010736261721393918
34: 0.0010647626101522696
35: 0.0010616460960959616
36: 0.0010593236885269326
37: 0.0010508778571339818
38: 0.0010513670217905996
39: 0.0010